In [ ]:
import pandas as pd
import numpy as np
import os

Delisted = pd.concat([pd.read_csv("Data/Delisted/"+path) for path in os.listdir("Data/Delisted/")], ignore_index=True)
Listed = pd.concat([pd.read_csv("Data/Listed/"+path) for path in os.listdir("Data/Listed/")], ignore_index=True)
Listing = pd.concat([pd.read_csv("Data/Listing/"+path) for path in os.listdir("Data/Listing/")], ignore_index=True)
Delisted["Code"] = [str(val) for val in Delisted["Code"]]
Listed["Code"] = [str(val) for val in Listed["Code"]]
Listing["Code"] = [str(val) for val in Listing["Code"]]

Delisted = Delisted[Delisted["Market Segment"]!="ETFs"].reset_index(drop=True)
Listed = Listed[Listed["Market Segment"]!="ETFs"].reset_index(drop=True)
Listing = Listing[Listing["Market Segment"]!="ETFs"].reset_index(drop=True)

Delisted = Delisted[Delisted["Market Segment"]!="ETNs"].reset_index(drop=True)
Listed = Listed[Listed["Market Segment"]!="ETNs"].reset_index(drop=True)
Listing = Listing[Listing["Market Segment"]!="ETNs"].reset_index(drop=True)

Listing.loc[Listing["Date of Listing"].isna(), "Date of Listing"] = Listing["Date of Listing (Date of Listing Approval)"][Listing["Date of Listing (Date of Listing Approval)"].notna()]
Listing.loc[Listing["Issue Name*2"].isna(), "Issue Name*2"] = Listing["Issue Name*3"][Listing["Issue Name*3"].notna()]

Listed.loc[Listed["Code"]=="45680", "Date of incorporation"] = "2005/09/28"
Listed.loc[Listed["Date of incorporation"]=="-", "Date of incorporation"] = "1800/01/01"

Listed["Date_of_incorporation"] = pd.to_datetime(Listed["Date of incorporation"]).dt.strftime("%Y-%m-%d")

def cut_string_dt(x):
    if len(x) > 13: return x[:13]
    return x

Listing["Date of Listing"] = Listing["Date of Listing"].apply(cut_string_dt)
Listing["Date of Listing"] = pd.to_datetime(Listing["Date of Listing"]).dt.strftime("%Y-%m-%d")
Delisted["Date"] = pd.to_datetime(Delisted["Date"]).dt.strftime("%Y")

In [ ]:
Listed.insert(0, "Time", 0)
dict_temp = {}
for year in range(2002, 2025):
    temp = Listed[Listed["Date_of_incorporation"]<=f"{year}-12-31"].copy()
    temp["Time"] = year
    temp["Date of Listing"] = pd.NA
    temp.sort_values("Date_of_incorporation", inplace=True)
    temp.reset_index(drop=True, inplace=True)
    dict_temp[year] = temp

In [ ]:
for i in range(len(Listing)):
    tmp_dict = Listing.loc[i].to_dict()
    year = int(tmp_dict["Date of Listing"][:4])
    tmp_dict["Issue name"] = tmp_dict["Issue Name*2"]
    tmp_dict["Code"] = str(tmp_dict["Code"])
    if tmp_dict["Code"] in dict_temp[year]["Code"].to_list():
        print(i)
        raise

    for Y in range(year, 2025):
        tmp_dict["Time"] = Y
        n = len(dict_temp[Y])
        dict_temp[Y].loc[n] = {}
        for col in dict_temp[Y].columns:
            if col in list(tmp_dict.keys()):
                dict_temp[Y].loc[n, col] = tmp_dict[col]

In [ ]:
dict_delist = {year: [] for year in dict_temp.keys()}
for i in range(len(Delisted)):
    date = int(Delisted.loc[i, "Date"])
    code = str(Delisted.loc[i, "Code"])
    for year in range(date, 2025):
        dict_delist[year].append(code)

for year in dict_delist.keys():
    start = len(dict_temp[year])
    dict_temp[year] = dict_temp[year][~dict_temp[year]["Code"].isin(dict_delist[year])]
    end = len(dict_temp[year])
    print(year, start - end)

In [ ]:
final_data = pd.concat(list(dict_temp.values()), ignore_index=True)
final_data.to_csv("All_history_listed.csv", index=False)

# Xử lý file Listing 0 và Listing 1 sai định dạng cột

In [ ]:
import pandas as pd

# Đường dẫn tới file CSV
file_path = 'Data/Listing/1.csv'

# Đọc hai dòng đầu tiên để lấy headers
with open(file_path, 'r') as file:
    header_line1 = file.readline().strip().split(',')
    header_line2 = file.readline().strip().split(',')

# Kết hợp hai dòng header lại với nhau
headers = header_line1 + header_line2
headers.__len__()

In [ ]:
data = pd.read_csv("Data/Listing/1.csv", skiprows=2, header=None)
data.__len__()

In [ ]:
list_rows = []
for i in range(0, len(data), 2):
    temp = list(data.loc[i]) + list(data.loc[i+1])
    list_rows.append(temp)

temp_df = pd.DataFrame(list_rows, columns=headers)
final_df = pd.DataFrame()
seen = []
for col in temp_df.columns:
    if col not in seen:
        seen.append(col)
        try:
            temp_df[col].shape[1]
            tmp_col = temp_df[col].iloc[:, 0]
        except:
            tmp_col = temp_df[col]
        
        final_df = pd.concat([final_df, tmp_col], axis=1)

final_df.shape

In [ ]:
final_df.to_csv("Data/Listing/1.csv", index=False)